In [13]:
MONK_TASK = 1
USER = "Nunzio"

In [ ]:
import sys
import os
if (colab := 'google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/Shareddrives/Project_ML_23/" + USER + "/machine-learning-project"
    sys.path.insert(0,BASE_PATH)
    sys.path.insert(0,BASE_PATH + "src/utils")
    !pip install optuna
    !pip install scikit-learn
    !pip install scikeras
    TRAIN_DATA = os.path.join(BASE_PATH, 'datasets', 'monk',f'monks-{MONK_TASK}.train')
    TEST_DATA = os.path.join(BASE_PATH, 'datasets', 'monk',f'monks-{MONK_TASK}.test')
    IMAGES_FOLDER = os.path.join(BASE_PATH, 'images', 'monk',  f'task-{MONK_TASK}', 'neural_network')
    MODEL_FOLDER = os.path.join(BASE_PATH, 'trained_models', 'monk', f'task-{MONK_TASK}')
else :   
    TRAIN_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.train')
    TEST_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.test')
    IMAGES_FOLDER = os.path.join('..', '..', '..', 'images', 'monk', f'task-{MONK_TASK}', 'neural_network')
    MODEL_FOLDER = os.path.join('..', '..', '..', 'trained_models', 'monk', f'task-{MONK_TASK}')

In [1]:
sys.path.append('../../utils')


import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set_theme(style='darkgrid')

from utils import save_plot

In [4]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 8))

df_train = pd.read_csv(TRAIN_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_test = pd.read_csv(TEST_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_train.head()

,1,2,3,4,5,6,7
0,1,1,1,1,1,3,1
1,1,1,1,1,1,3,2
2,1,1,1,1,3,2,1
3,1,1,1,1,3,3,2
4,1,1,1,2,1,2,1


In [5]:
features = ['feature_' + str(i) for i in range(1, 7)]

# Rename columns
new_column_names = ['class'] + features

df_train.columns = new_column_names
df_test.columns = new_column_names

df_train.head()

,class,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,1,1,1,1,1,3,1
1,1,1,1,1,1,3,2
2,1,1,1,1,3,2,1
3,1,1,1,1,3,3,2
4,1,1,1,2,1,2,1


In [6]:
df_train_encoded = pd.get_dummies(df_train, columns=features)
df_test_encoded = pd.get_dummies(df_test, columns=features)

df_train_encoded, df_test_encoded = df_train_encoded.align(df_test_encoded, join='inner', axis=1)

df_train_encoded.head()

,class,feature_1_1,feature_1_2,feature_1_3,feature_2_1,feature_2_2,feature_2_3,feature_3_1,feature_3_2,feature_4_1,feature_4_2,feature_4_3,feature_5_1,feature_5_2,feature_5_3,feature_5_4,feature_6_1,feature_6_2
0,1,True,False,False,True,False,False,True,False,True,False,False,False,False,True,False,True,False
1,1,True,False,False,True,False,False,True,False,True,False,False,False,False,True,False,False,True
2,1,True,False,False,True,False,False,True,False,False,False,True,False,True,False,False,True,False
3,1,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,False,True
4,1,True,False,False,True,False,False,False,True,True,False,False,False,True,False,False,True,False


In [7]:
features = df_train_encoded.columns.difference(['class'])

X_train = df_train_encoded[features].to_numpy()
y_train = df_train_encoded['class'].to_numpy()

X_test = df_test_encoded[features].to_numpy()
y_test = df_test_encoded['class'].to_numpy()

# Create model

In [8]:
from keras.src.regularizers import l2
from keras.src.optimizers import Adam, SGD
from keras.models import Sequential
from keras.layers import Dense

def create_model(architecture=(8,), optimizer='adam', learning_rate=0.001, lamda=0.1, alpha=0.9):
    model = Sequential()
    model.add(Dense(units=architecture[0], input_dim=X_train.shape[1], activation='relu',
                    kernel_regularizer=l2(lamda)))

    for units in architecture[1:]:
        model.add(Dense(units=units, activation='relu', kernel_regularizer=l2(lamda)))

    model.add(Dense(1, activation='sigmoid'))

    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate, momentum = alpha)
    else:
        opt = SGD(learning_rate=learning_rate, momentum = alpha)

    model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
    return model

In [9]:
from scikeras.wrappers import KerasClassifier

model = KerasClassifier(create_model, verbose=0)

In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'model__architecture': [
        (8), (8, 8), (8, 8, 8)
    ],
    'model__optimizer': ['adam', 'sgd'],
    'model__learning_rate': [0.3],
    'model__alpha': [0.9],
    'model__lamda': [0.1],
    'epochs': [200],
    'batch_size': [8,16,32]
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy'
)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


2024-01-14 16:07:07.872489: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-14 16:07:07.872615: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-14 16:07:07.872636: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-14 16:07:07.873732: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-14 16:07:07.874650: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-01-14 16:07:07.908801: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-14 16:07:07.908866: I metal_plugin/src/device/metal_device.cc:296] system

[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=8, model__lamda=0.1, model__learning_rate=0.3, model__optimizer=sgd; total time=   0.2s
[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=8, model__lamda=0.1, model__learning_rate=0.3, model__optimizer=adam; total time=   0.3s
[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=8, model__lamda=0.1, model__learning_rate=0.3, model__optimizer=adam; total time=   0.2s


2024-01-14 16:07:08.368294: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-14 16:07:08.369984: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-14 16:07:08.370053: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-14 16:07:08.370264: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-14 16:07:08.370426: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=8, model__lamda=0.1, model__learning_rate=0.3, model__optimizer=adam; total time=   0.3s


...

[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=8, model__lamda=0.1, model__learning_rate=0.3, model__optimizer=sgd; total time=   0.5s
[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=8, model__lamda=0.1, model__learning_rate=0.3, model__optimizer=sgd; total time=   0.4s


2024-01-14 16:07:09.227568: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-01-14 16:07:09.228770: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-14 16:07:09.228850: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-14 16:07:09.229052: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-14 16:07:09.229127: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=8, model__lamda=0.1, model__learning_rate=0.3, model__optimizer=adam; total time=   0.4s
[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=(8, 8), model__lamda=0.1, model__learning_rate=0.3, model__optimizer=adam; total time=   5.8s
[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=(8, 8), model__lamda=0.1, model__learning_rate=0.3, model__optimizer=adam; total time=   5.9s
[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=(8, 8), model__lamda=0.1, model__learning_rate=0.3, model__optimizer=adam; total time=   5.6s
[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=(8, 8), model__lamda=0.1, model__learning_rate=0.3, model__optimizer=adam; total time=   5.4s
[CV] END batch_size=8, epochs=200, model__alpha=0.9, model__architecture=(8, 8), model__lamda=0.1, model__learning_rate=0.3, model__optimizer=adam; total time=   6.5s
[CV

2024-01-14 16:07:19.407450: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-01-14 16:07:19.630981: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-01-14 16:07:19.784693: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-01-14 16:07:20.329383: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-01-14 16:07:20.358949: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-01-14 16:07:20.372760: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-01-14 16:07:20.651440: I tensorflow/core/grappler/optimizers/cust

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

final_model = grid_search.best_estimator_
y_pred = final_model.predict(X_test)

print('Best parameters: ', grid_search.best_params_)
print('Best accuracy: ', grid_search.best_score_)
print('Test set accuracy: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Learning curve

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, validation_scores = learning_curve(
    estimator=final_model,
    X=X_train,
    y=y_train,
    train_sizes=[0.1, 0.33, 0.55, 0.78, 1.],
    cv=5,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy'
)

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
validation_scores_mean = np.mean(validation_scores, axis=1)
validation_scores_std = np.std(validation_scores, axis=1)

In [ ]:
plt.figure()
plt.title('Learning curve', fontweight='bold', fontsize=16)
plt.xlabel('Training examples', fontweight='bold')
plt.ylabel('Score', fontweight='bold')
plt.grid(True)

color1 = sns.dark_palette((20, 60, 50), input='husl')[-1]
color2 = sns.dark_palette('seagreen')[-1]

# Filling the area around the mean scores to indicate variability of the model's performance
# The shaded area represents the range of scores (mean ± standard deviation) for each training set size
plt.fill_between(
    train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2, color=color1
)
plt.fill_between(
    train_sizes, validation_scores_mean - validation_scores_std,
                 validation_scores_mean + validation_scores_std, alpha=0.2, color=color2
)

# Mean score lines for training and validation
sns.lineplot(x=train_sizes, y=train_scores_mean, marker='s', color=color1, label='Training score')
sns.lineplot(x=train_sizes, y=validation_scores_mean, marker='s', color=color2, label='Cross-validation score')

plt.legend(loc='lower right')

save_plot(plt, IMAGES_FOLDER, 'learning_curve')
plt.show()

# Save model

In [ ]:
from joblib import dump

model_path = os.path.join(MODEL_FOLDER, 'NN_model.joblib')
dump(final_model, model_path, compress=3)